# Process CCP simulation results - UTC Microgrid

In [ ]:
import pandas as pd
import numpy as np
import os, fnmatch
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings('ignore', category=DeprecationWarning)
import glob
import seaborn as sns
import gzip
import matplotlib.style as style
from matplotlib.path import Path
from matplotlib.patches import BoxStyle
from sys import platform
import pyarrow as pa
%matplotlib inline

In [ ]:
# Import custom python file from github repo: https://changhsinlee.com/colab-import-python/
!pip install requests
import requests
# Save python as file to colab working directory
# If you are using GitHub, make sure you get the "Raw" version of the code
url = 'https://raw.githubusercontent.com/levorato/ccp_rtcs/master/notebooks/rccp_utils.py'
r = requests.get(url)
# make sure your filename is the same as how you want to import 
with open('rccp_utils.py', 'w') as f:
    f.write(r.text)
# now we can import
from rccp_utils import *

## 1. Process result files

### 1.1. Setup project folders

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive/')

Mounted at /content/gdrive/


In [ ]:
gdrive_folder = '/content/gdrive/MyDrive'
project_folder = '..'
antoine_instances_folder = os.path.join(project_folder, "instances", "utc_skew")
toy_instances_folder = os.path.join(project_folder, "instances", "toy")
instances_folder = os.path.join(project_folder, "instances")
japan_instances_folder = os.path.join(project_folder, "instances", "japan_microgrid")
output_folder = os.path.join(gdrive_folder, "rccp_experiments")
results_folder = os.path.join(gdrive_folder, "rccp_results")
print("*** Project folder is", project_folder)
print("*** Instances folder is",  instances_folder)
print("*** Output folder is", output_folder)


*** Project folder is ..
*** Instances folder is ../instances
*** Output folder is /content/gdrive/MyDrive/rccp_experiments


### 1.2. List which experiments to process

In [ ]:
experiment_list = ["run_sim"]

In [ ]:
experiment_folder_list = [os.path.join(output_folder, exp) for exp in experiment_list]
experiment_folder_list

['/content/gdrive/MyDrive/rccp_experiments/run_sim']

### 1.3. List which CPP models to process

In [ ]:
simulated_model_list = ["robust-budget", "robust-box", "robust-budget"]

### 1.4. Select instance_group to process

In [ ]:
instance_group_list = ["utc-skew"]

### 1.5. Select RTCS forecast types to process

In [ ]:
forecast_type_list = ["average"]  # average-based RTCS forecast

In [ ]:
instances_to_process = ["A_instance2_1000s_skewed-left.txt", "A_instance2_1000s_skewed-right.txt", "A_instance2_1000s_uniform.txt"]
instance_group = "utc-skew"
instances_to_process

['A_instance2_1000s_skewed-left.txt',
 'A_instance2_1000s_skewed-right.txt',
 'A_instance2_1000s_uniform.txt']

In [ ]:
df_list = []
for instance_ in instances_to_process:
    result_path = os.path.join(experiment_folder_list[0], "output", "simulation", "zip", instance_name)
    df_ = read_concatenated_trace_df(result_path)
    df_['InstanceName'] = instance_name
    df_list.append(df_)
df = pd.concat(df_list)

Looking for existing simulations in folder /content/gdrive/MyDrive/rccp_experiments/run_sim/output/simulation/zip/A_instance2_1000s_skewed-left.txt ...
Concatenating individual trace_df dataframes...
Concatenation done.
Looking for existing simulations in folder /content/gdrive/MyDrive/rccp_experiments/run_sim/output/simulation/zip/A_instance2_1000s_skewed-left.txt ...
Concatenating individual trace_df dataframes...
Concatenation done.
Looking for existing simulations in folder /content/gdrive/MyDrive/rccp_experiments/run_sim/output/simulation/zip/A_instance2_1000s_skewed-left.txt ...
Concatenating individual trace_df dataframes...
Concatenation done.


In [ ]:
df.head()

,Model,GammaPerc,Gamma,Strategy,Reoptimize,ModelPolicy,ForecastType,ScenarioId,t,d,ObjValue,OptTimeSpent,e_td,gap,cost,RealProcTime,InstanceName
0,deterministic,0.0,0.0,conservative,True,full_model,average,251,1,1,7044.40164,0.75382,0.0,0.0,25.839632,0.001015,A_instance2_1000s_skewed-left.txt
1,deterministic,0.0,0.0,conservative,True,full_model,average,251,1,2,7044.40164,0.00000,0.0,0.0,24.941374,0.001700,A_instance2_1000s_skewed-left.txt
2,deterministic,0.0,0.0,conservative,True,full_model,average,251,1,3,7044.40164,0.00000,0.0,0.0,25.163126,0.002408,A_instance2_1000s_skewed-left.txt
3,deterministic,0.0,0.0,conservative,True,full_model,average,251,1,4,7044.40164,0.00000,0.0,0.0,25.163126,0.003108,A_instance2_1000s_skewed-left.txt
4,deterministic,0.0,0.0,conservative,True,full_model,average,251,1,5,7044.40164,0.00000,0.0,0.0,25.163126,0.003785,A_instance2_1000s_skewed-left.txt


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2332800 entries, 0 to 143
Data columns (total 17 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Model         object 
 1   GammaPerc     float64
 2   Gamma         float64
 3   Strategy      object 
 4   Reoptimize    bool   
 5   ModelPolicy   object 
 6   ForecastType  object 
 7   ScenarioId    int64  
 8   t             int64  
 9   d             int64  
 10  ObjValue      float64
 11  OptTimeSpent  float64
 12  e_td          float64
 13  gap           float64
 14  cost          float64
 15  RealProcTime  float64
 16  InstanceName  object 
dtypes: bool(1), float64(8), int64(3), object(5)
memory usage: 304.8+ MB


### Create the output folders for processed results

In [ ]:
reportfolder = os.path.join(results_folder, 'consolidated_results', 'df')
if not os.path.exists(reportfolder):
    os.makedirs(reportfolder)
print('Saving files on folder: ' + reportfolder)

Saving files on folder: /content/gdrive/MyDrive/rccp_results/consolidated_results/df


### Export consolidated results dataframe to pickle file


In [ ]:
pickle_filepath = os.path.join(reportfolder, experiment_list[0] + '.' + instance_group + '.' + 'A_instance2.results.pkl.gz')
df.to_pickle(pickle_filepath)
print('Saved consolidated results to: ', pickle_filepath)

Saved consolidated results to:  /content/gdrive/MyDrive/rccp_results/consolidated_results/df/run_sim.utc-skew.A_instance2.results.pkl.gz
